# Eigenvalue problems

We solve the generalized eigenvalue problem

$$
A u = \lambda M u,
$$

where $A$ comes from $\int \nabla u \nabla v$, and $M$ from $\int u v$, on the space $H_0^1$

In [3]:
import netgen.gui
%gui tk
from netgen.geom2d import unit_square
from ngsolve import *
from math import pi
import scipy.linalg
from scipy import random 
from random import random

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

# setup stiffness and mass matrix:
fes = H1(mesh, order=4, dirichlet=[1,2,3,4])
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))
pre = Preconditioner(a, "multigrid")

m = BilinearForm(fes)
m += SymbolicBFI(u*v)

a.Assemble()
m.Assemble()

u = GridFunction(fes)

The inverse iteration is
$$
u_{n+1} = A^{-1} M u_n,
$$
with the Rayleigh quotient 
$$
\rho_n = \frac{(A u_n, u_n)}{(M u_n, u_n)}
$$

It converges to the smallest eigen-value, with rate of convergence $\lambda_1 / \lambda_2$.

The preconditioned inverse iteration (PINVIT), see [Knyazef+Neymeyr], replaces $A^{-1}$ by an approximate inverse $C^{-1}$:

$$
\rho_n = \frac{(A u_n, u_n)}{(M u_n, u_n)} \\
w_n = C^{-1} (A u_n - \rho M u_n) \\
u_{n+1} = u_n + \alpha w_n
$$

The optimal step-size $\alpha$ is found by minimizing the Rayleigh-quotient on a two-dimensional space:

$$
u_{n+1} = \operatorname{arg} \min_{v \in \{ u_n, w_n\}} \frac{(A v, v)}{(M v, v)} 
$$

The minimization problem can be solved by an small eigenvalue problem

$$
a y = \lambda m y
$$
with matrices
$$
a = 
\left( \begin{array}{cc}
\left< A u_n, u_n \right> &
\left< A u_n, w_n \right> \\
\left< A w_n, u_n \right> &
\left< A w_n, w_n \right> 
\end{array} \right) \quad
m = 
\left( \begin{array}{cc}
\left< M u_n, u_n \right> &
\left< M u_n, w_n \right> \\
\left< M w_n, u_n \right> &
\left< M w_n, w_n \right> 
\end{array} \right)
$$

Then, the new iterate is

$$
u_{n+1} = y_1 u_n + y_2 w_n
$$

where $y$ is the eigenvector corresponding to the smaller eigenvalue.

### Implementation in NGSolve

Create some help vectors:

In [10]:
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()

random initial conditions, zero on the boundary

In [8]:
freedofs = fes.FreeDofs()    
for i in range(len(u.vec)):
    u.vec[i] = random() if freedofs[i] else 0   

In [6]:
for i in range(20):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotiont
    lam = auu/muu
    print (lam / (pi**2))
    # residual
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)
    
    
    eval,evec = scipy.linalg.eigh(asmall, b=msmall)
    # print (eval / pi**2)
    # print (evec)
    u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w
    
Draw (u)

23.630935563629585
2.081558863859152
2.0011403688039535
2.000112490165907
2.000020128831872
2.000004045057324
2.000000837554281
2.000000175792457
2.0000000372504987
2.000000008001936
2.000000001754522
2.0000000004105822
2.0000000001184888
2.0000000000546154
2.0000000000405245
2.0000000000374007
2.000000000036703
2.000000000036545
2.000000000036512
2.0000000000365032


## Simultaneous iteration for several eigenvalues

Declare GridFunction with multiple components:

In [ ]:
num = 5
u = GridFunction(fes, multidim=num)

Create list of help vectors:

In [ ]:
r = u.vec.CreateVector()
Av = u.vec.CreateVector()
Mv = u.vec.CreateVector()

vecs = []
for i in range(2*num):
    vecs.append (u.vec.CreateVector())

freedofs = fes.FreeDofs()    
for v in u.vecs:
    for i in range(len(u.vec)):
        v[i] = random() if freedofs[i] else 0

Compute num residuals, and solve small eigenvalue problem on $2 num$-dimensional space

In [12]:
lams = num * [1]

asmall = Matrix(2*num, 2*num)
msmall = Matrix(2*num, 2*num)

for i in range(100):
    
    for j in range(num):
        vecs[j].data = u.vecs[j]
        r.data = a.mat * vecs[j] - lams[j] * m.mat * vecs[j]
        # r.data = 1/Norm(r) * r
        r *= 1/Norm(r)
        vecs[num+j].data = pre.mat * r

    for j in range(2*num):
        Av.data = a.mat * vecs[j]
        Mv.data = m.mat * vecs[j]
        for k in range(2*num):
            asmall[j,k] = InnerProduct(Av, vecs[k])
            msmall[j,k] = InnerProduct(Mv, vecs[k])

    ev,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    lams[0:num] = ev[0:num]
    print (i, ":", [lam/pi**2 for lam in lams])
    
    for j in range(num):
        r[:] = 0.0
        for k in range(2*num):
            r.data += float(evec[k,j]) * vecs[k]
        u.vecs[j].data = r
    
Draw (u)

0 : [10.591414903685335, 70.546343581482049, 81.240642540018541, 95.846793919799254, 105.74611486190483]
1 : [2.045298440010249, 6.2867337615046681, 10.739815773483752, 13.874831263892393, 19.764487311795268]
2 : [2.0016980285178421, 5.0594345747061125, 5.4227674972640489, 8.3296431446347938, 11.256592296316622]
3 : [2.0002477226117756, 5.0040040041771761, 5.0267256021274429, 8.063690286965123, 10.357330514534656]
4 : [2.0000473663636855, 5.0003866789974829, 5.0021914068614688, 8.0147294548396637, 10.130420760588391]
5 : [2.0000097333478566, 5.0000522606815316, 5.0002429927564211, 8.0037176937795547, 10.051096771192546]
6 : [2.0000020418129858, 5.0000092648531194, 5.0000348225192166, 8.0010486438596313, 10.020293424491767]
7 : [2.0000004349136029, 5.0000018651132203, 5.0000062088139492, 8.0003366037155246, 10.008097173687434]
8 : [2.000000093838346, 5.0000004028783751, 5.0000012413006818, 8.0001234407912882, 10.003235874649318]
9 : [2.0000000204466239, 5.0000000923171708, 5.00000026677